In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/all-animes-in-mal/mal_anime.csv


In [2]:
anime=pd.read_csv("/kaggle/input/all-animes-in-mal/mal_anime.csv")

In [3]:
anime.head()

,Unnamed: 0,title,episodes,status,theme,demographic,genres,type,favorites,popularity,rank,score,members,synopsis,aired,duration,premiered,studios
0,0,Fullmetal Alchemist: Brotherhood,64,Finished Airing,Military,Shounen,"Action,Adventure,Drama,Fantasy,",TV,"218,277",#3,#1,9.10,"3,190,961",After a horrific alchemy experiment goes wrong...,"Apr 5, 2009 to Jul 4, 2010",24 min. per ep.,Spring 2009,Bones
1,1,Steins;Gate,24,Finished Airing,Unknown,Unknown,"Drama,Sci-Fi,Suspense,",TV,"183,596",#13,#2,9.07,"2,452,142",Eccentric scientist Rintarou Okabe has a never...,"Apr 6, 2011 to Sep 14, 2011",24 min. per ep.,Spring 2011,White Fox
2,2,Bleach: Sennen Kessen-hen,13,Finished Airing,Unknown,Shounen,"Action,Adventure,Fantasy,",TV,"18,421",#458,#3,9.06,"455,428",Substitute Soul Reaper Ichigo Kurosaki spends ...,"Oct 11, 2022 to Dec 27, 2022",24 min. per ep.,Fall 2022,Pierrot
3,3,Gintama°,51,Finished Airing,Unknown,Shounen,"Action,Comedy,Sci-Fi,",TV,"16,042",#332,#4,9.06,"599,235","Gintoki, Shinpachi, and Kagura return as the f...","Apr 8, 2015 to Mar 30, 2016",24 min. per ep.,Spring 2015,Bandai Namco Pictures
4,4,Kaguya-sama wa Kokurasetai: Ultra Romantic,13,Finished Airing,School,Seinen,"Comedy,Romance,",TV,"29,397",#193,#5,9.05,"832,346",The elite members of Shuchiin Academy's studen...,"Apr 9, 2022 to Jun 25, 2022",23 min. per ep.,Spring 2022,A-1 Pictures


In [4]:
anime.rename(columns={"Unnamed: 0": "index"}, inplace=True)

In [5]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24262 entries, 0 to 24261
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        24262 non-null  int64  
 1   title        24262 non-null  object 
 2   episodes     24262 non-null  object 
 3   status       24262 non-null  object 
 4   theme        24262 non-null  object 
 5   demographic  24262 non-null  object 
 6   genres       24262 non-null  object 
 7   type         24262 non-null  object 
 8   favorites    24262 non-null  object 
 9   popularity   24262 non-null  object 
 10  rank         20197 non-null  object 
 11  score        15294 non-null  float64
 12  members      24262 non-null  object 
 13  synopsis     24262 non-null  object 
 14  aired        24262 non-null  object 
 15  duration     24262 non-null  object 
 16  premiered    24262 non-null  object 
 17  studios      24262 non-null  object 
dtypes: float64(1), int64(1), object(16)
memory usa

In [6]:
!pip install summarytools
from summarytools import dfSummary
dfSummary(anime)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for summarytools: filename=summarytools-0.2.3-py3-none-any.whl size=8577 sha256=fcedfd7a3350be65b2bbe6c4a767f660f4beba06c4641d1f4e593c66a0f1e833
  Stored in directory: /root/.cache/pip/wheels/d5/8a/5d/008e2127a7f9dd64d066f16e5ea268fa52823dc929c982d6f1
Successfully built summarytools


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,index[int64],Mean (sd) : 12130.5 (7004.0)min < med < max:0.0 < 12130.5 < 24261.0IQR (CV) : 12130.5 (1.7),"24,262 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAABxklEQVR4nO3XQU7CYBBA4fkbJSCkDSHCAbwBSw7hYT0EJ2DhxhU7EtKUUDEarCcAKTXqS963baczixcSUtM0If132V8fIF3i5rsXUkqDiOhd+f0sIj6vnO06T91Nvbvr/HvTNIdTD8+GmlIaTCaTxzzPx223Ho/H27quH0aj0UuWZR+/OU/dTb37J+Z3u12ZUno6Fet3v6i9PM/Hi8XiUBTFW5vF6/V6vFqt7ufz+fNsNivbzHadp+6m3t11vqqq/nK5HG+3215EXBVqREQURfE2nU5f2ywvy3IQETEcDlvPdp2n7qbe/RPzETE499A/U0IwVCEYqhAMVQiGKgRDFYKhCsFQhWCoQjBUIRiqEAxVCIYqBEMVgqEKwVCFYKhCMFQhGKoQDFUIhioEQxWCoQrBUIVgqEIwVCEYqhAMVQiGKgRDFYKhCsFQhWCoQjBUIRiqEAxVCIYqBEMVgqEKwVCFYKhCMFQhGKoQDFUIhioEQxWCoQrBUIVgqEIwVCEYqhAMVQiGKgRDFYKhCsFQhWCoQjBUIRiqEAxVCIYqBEMVgqEK4eaSl6qq6rf98H6/70dE1HXd32w2d785T91Nvbvr/CV9fQGSljnmc7n3zQAAAABJRU5ErkJggg=="">",0(0.0%)
2,title[object],1. Crayon Shin-chan Movie 03: Unk2. Kimi no Na wo Yobeba3. Man Jie Guoxue Xilie: Chengyu 4. Man Jie Guoxue Xilie: Yuyan Gu5. Man Shu Lan Liuliu6. Man with a Mission Animated Sh7. Man Youqu8. Manabu no Natsuyasumi9. Ima10. Ima demo Senchou to Yobarete I11. other,"2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)2 (0.0%)24,242 (99.9%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAADLklEQVR4nO3cwY3TYBRG0ecogmFja2QpJUAPKYItvdOFNzCswoIeriVzTgX/4kpPkfxleb1eA4Xb2Q/g/yE2MmIjIzYyYiMjNjJiIyM2MmIjIzYyt2VZvi3L8nb2Q7i+28x8nZnPZz+E63NGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3MbWZ+zsyfsx/C9S3+xYiKM0pGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kDF4IeOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEZGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKBmxkREbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYy1lVkrKvIOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKJllZr7MzKezH8L13fd9/76u6/vZD+H67uu6vj+fz9/btn2c/Riu6ziOt/vMzLZtH4/H49fZD+La/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjc5/59/nH2Q/h2o7jeFv2ff/h40kKPgsnY11Fxg8EMmIjIzYyYiMjNjJiIyM2MmIjIzYyfwFj50uCzHpkKgAAAABJRU5ErkJggg=="">",0(0.0%)
3,episodes[object],1. 12. 123. 24. 265. 136. 527. 38. 49. Unknown10. 611. other,"11,269 (46.4%)1,935 (8.0%)1,308 (5.4%)1,199 (4.9%)990 (4.1%)808 (3.3%)657 (2.7%)522 (2.2%)520 (2.1%)408 (1.7%)4,646 (19.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjcuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8pXeV/AAAACXBIWXMAAA9hAAAPYQGoP6dpAAAEO0lEQVR4nO3dwU4TbQCG0alpiG7aEJJeTC/Ci/UiejcNaRdi3PRf/GpACkiZeUroOTtXTszjR2byvTo7HA4DFD6d+wG4HGIjIzYyYiMjNjJiIyM2MmIjIzYys2EYvgzDcHXuB3nHfh4Oh7tzP8RHML+5ufm6WCyuz/0g79V+v7+dzWbfBPd288Vicb1er++Wy+WPcz/Me7Pb7T5vNpvr7XZ7NQyD2N5oPgzDsFwuf6xWq+/nfph36su5H+Cj8IJARmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxk5sPw/72tcz/Ie+TPZVzz/X5/u9lsrgf3to7a7/e3wzD8PPdzfAQ2CC+zQRjJzD+ZReWUk83fdE7y6nWVtRGnetW6ytqItzhlXeWtlZP4qEtGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbmVYMXAxDe4tWDFwMQTuVaOBmDFzL/crI5yRjFi4MXAxfG8uzgxcCFMf3L4MXAhVH4qEtGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbm2cGLgQtjenHwYuDCWFwLJ2PwQsYLAhk/RslYV5GxriJjXUXGCwIZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZ6yoy1lVkXAsnY11F5rmTzYnGqJ4cvBi6MLajgxdDF6bw3ODF0IVR+ahLRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5ujgxdCFKTw5eDF0YWyuhZMxeCHz1MnmVGN0Rwcvxi5M4dHgxdiFqTw1eDF2YXQ+6pIRGxmxkREbGbGRERsZsZER

In [7]:
features=['genres','theme','demographic','synopsis','type']
print(features)

['genres', 'theme', 'demographic', 'synopsis', 'type']


In [8]:
for ani in features:
    anime[ani]=anime[ani].fillna('')
   

In [9]:
combined_features=anime['genres']+' '+anime['theme']+' '+anime['demographic']+' '+anime['synopsis']+' '+anime['type']
print(combined_features)

0        Action,Adventure,Drama,Fantasy, Military Shoun...
1        Drama,Sci-Fi,Suspense, Unknown Unknown Eccentr...
2        Action,Adventure,Fantasy, Unknown Shounen Subs...
3        Action,Comedy,Sci-Fi, Unknown Shounen Gintoki,...
4        Comedy,Romance, School Seinen The elite member...
                               ...                        
24257    Hentai, Unknown Unknown In which the act of ra...
24258    Hentai, Unknown Unknown Based off the eroge by...
24259    Hentai, Unknown Unknown Sister Rosaria is work...
24260    Hentai, Unknown Unknown There are nine men and...
24261    Drama,Romance,Hentai, Unknown Unknown Yusuke's...
Length: 24262, dtype: object


In [10]:
vectorizer=TfidfVectorizer()

In [11]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [12]:
similarity=cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.07976103 0.07649965 ... 0.06869925 0.03861332 0.04281262]
 [0.07976103 1.         0.09168122 ... 0.10087775 0.03525027 0.09895145]
 [0.07649965 0.09168122 1.         ... 0.05723554 0.01816791 0.06337459]
 ...
 [0.06869925 0.10087775 0.05723554 ... 1.         0.06631765 0.07055901]
 [0.03861332 0.03525027 0.01816791 ... 0.06631765 1.         0.048456  ]
 [0.04281262 0.09895145 0.06337459 ... 0.07055901 0.048456   1.        ]]


In [13]:
anime_name=input("enter anime name: ")
list_of_all_titles = anime['title'].tolist()

find_close_match = difflib.get_close_matches(anime_name, list_of_all_titles)
print(find_close_match)
close_match = find_close_match[0]

index_of_the_anime = anime[anime.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_anime]))

sorted_similar_anime = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('anime suggested for you : \n')

i = 1

for anim in sorted_similar_anime:
    index = anim[0]
    title_from_index = anime[anime.index==index]['title'].values[0]
    if (i<30):
        print(i, '.',title_from_index)
        i+=1

enter anime name:  one piece


['One Piece', 'One Piece 4D', 'Ange Vierge']
anime suggested for you : 

1 . One Piece
2 . One Piece: Episode of East Blue - Luffy to 4-nin no Nakama no Daibouken
3 . One Piece Movie 14: Stampede
4 . One Piece Film: Z
5 . One Piece: Romance Dawn Story
6 . One Piece Film: Strong World Episode 0
7 . One Piece Film: Strong World
8 . One Piece: Oounabara ni Hirake! Dekkai Dekkai Chichi no Yume!
9 . Kaizoku Ouji
10 . One Piece 3D2Y: Ace no shi wo Koete! Luffy Nakama Tono Chikai
11 . Mashou no Nie 3
12 . Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare
13 . Shironeko Project: Zero Chronicle
14 . One Piece Film: Red
15 . One Piece Movie 08: Episode of Alabasta - Sabaku no Oujo to Kaizoku-tachi
16 . One Piece Movie 01
17 . Helck
18 . One Piece Movie 04: Dead End no Bouken
19 . Rennyo to Sono Haha
20 . Shen Mu
21 . Doraemon Movie 19: Nobita no Nankai Daibouken
22 . Tokyo One Piece Tower: Tongari Shima no Dai Hihou
23 . One Piece: Episode of Sabo - 3 Kyoudai no Kizuna Kiseki no Saikai to Uk